In [4]:
#TWO AMBULANCE DISTANCE MINIMISATION - 

import numpy as np 
#RC utility functions    
from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateTwoAmbulanceAdjacencyV1,CreateTwoAmbulanceAdjacencyV2
#################################### Create Adjacency defined as a qubo ####################################
n_destinations = 4  
gridWidth = 2# n_destinations
Width  = gridWidth
Height = n_destinations//gridWidth

result = CreateTwoAmbulanceAdjacencyV2(gridWidth,n_destinations, Adddistance = 1,use_XYMixer_constraints=0)

Adjacency = result['qubo']
AdjacencyHybrid = result['quboHybrid']
n_qubits = result['n_qubits']
ConstraintMultiplier = result['ConstraintMultiplier']
max_distance = result['max_distance']

if 1:
    filename = 'Twoambulances.txt'
    print_QUBOdetails(Adjacency,n_qubits,filename)

ADJACENCY MATRIX Twoambulances.txt  

Qubit  q0   q1   q2   q3   q4   q5   q6   q7   q8   q9  q10  q11  q12  q13  q14  q15 

  q0  -20                  40                       1    1    2  

  q1       -20                  40             1         2    1  

  q2            -20                  40        1    2         1  

  q3                 -20                  40   2    1    1  

  q4                      -20                                           1    1    2  

  q5                           -20                                 1         2    1  

  q6                                -20                            1    2         1  

  q7                                     -20                       2    1    1  

  q8                                          -20   40   40   40 

  q9                                               -20   40   40 

 q10                                                    -20   40 

 q11                                                         -20 

 

In [57]:
import dimod 
Myqubo = dimod.BinaryQuadraticModel.from_qubo(Adjacency)
samplerExact = dimod.ExactSolver()
response = samplerExact.sample(Myqubo) 

print(response.truncate(3,sorted_by='energy',), '\nOrder q0...qn, samples ordered by energy')

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0  0  1  1  1  1  0  0  0  0  0  0  1  1  0  0  0 -118.0       1
1  0  0  1  1  1  1  0  0  0  0  0  1  0  1  0  0 -118.0       1
2  1  1  0  0  0  0  1  1  1  0  0  0  0  0  1  0 -118.0       1
['BINARY', 3 rows, 3 samples, 16 variables] 
Order q0...qn, samples ordered by energy


In [87]:
response1 = samplerExact.sample(Myqubo) 
n_sub_samples = 3
print(response1.truncate(n_sub_samples,sorted_by=None), '\nOrder of each sample q0...qn. First ',n_sub_samples, ' UNORDERED samples, then ranked by energy')

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  -40.0       1
1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  -20.0       1
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0    0.0       1
['BINARY', 3 rows, 3 samples, 16 variables] 
Order of each sample q0...qn. First  3  UNORDERED samples, then ranked by energy


In [66]:
print(list( response.first[0].values()), ' = Lowest energy sample as a list...')
print(      response.first[0],         ' ... or a dict')
n=4
print(list( response.samples()[n].values() ) ,'  ',n,'th sample as a list')

[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]  = Lowest energy sample as a list...
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 1, 13: 0, 14: 0, 15: 0}  ... or a dict
[1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0]    4 th sample as a list


In [67]:
print(response.first[1], 'Lowest energy of all data in the response')
print(list( response.first[0].values()), ' = Lowest energy sample as a list...')

-118.0 Lowest energy of all data in the response
[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]  = Lowest energy sample as a list...


In [65]:
print(response.data_vectors['energy'], 'List of  energies UNORDERED')

[  0. -20. -40. ... -60. -40. -20.] List of  energies UNORDERED


In [85]:
print('UNPACK each sample from the sampleSet, in ORDER of (energy, or of evaluations)  into\n', 'Sample', '\t\t\t\t\t\tenergy','\tnum_occurrences', 'evaluation order')
for n,datum in enumerate( response.data(fields=['sample', 'energy','num_occurrences'], sorted_by=None)):     #'energy'
    if n<10:
        print(list(datum.sample.values() ), '\t', datum.energy ,'\t', datum.num_occurrences,'\t',n,'\t= n')

UNPACK each sample from the sampleSet, in ORDER of (energy, or of evaluations)  into
 Sample 						energy 	num_occurrences evaluation order
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 0.0 	 1 	 0 	= n
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -20.0 	 1 	 1 	= n
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -40.0 	 1 	 2 	= n
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -20.0 	 1 	 3 	= n
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -40.0 	 1 	 4 	= n
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -60.0 	 1 	 5 	= n
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -40.0 	 1 	 6 	= n
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -20.0 	 1 	 7 	= n
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -40.0 	 1 	 8 	= n
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 	 -60.0 	 1 	 9 	= n
